In [ ]:
!pip3 install pandas
!pip3 install numpy
!pip3 install scikit-learn

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")

# Elimina todas las variables meteorológicas que no correspondan a la localización de Sotavento (la localización 13)
wind_ava = wind_ava.filter(regex='^(datetime|energy|.*\.13)$')

# Elimina las variables 'lai_hv.13' y 'v10n.13'
if 'lai_hv.13' in wind_ava.columns and 'v10n.13' in wind_ava.columns:
    wind_ava = wind_ava.drop(['lai_hv.13', 'v10n.13'], axis=1)

# Vuelve a definir las variables numéricas
numerical_vars = wind_ava.select_dtypes(include=['int64', 'float64']).columns

# Quitamos "energy" de las variables numéricas, ya que querremos usarla como variable objetivo
numerical_vars = numerical_vars.drop('energy')

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(wind_ava[numerical_vars], wind_ava['energy'], test_size=1/3, random_state=100472166, shuffle=False)

# Crear el escalador
scaler = StandardScaler()

# Ajustar el escalador a las características del conjunto de entrenamiento y transformarlas
X_train_scaled = scaler.fit_transform(X_train)

# Transformar las características del conjunto de prueba usando el mismo escalador
X_test_scaled = scaler.transform(X_test)

# Dividir los datos de entrenamiento en un conjunto de entrenamiento más pequeño y un conjunto de validación
X_train_inner, X_val, y_train_inner, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=100472166)

# Definir el modelo SVM con los hiperparámetros seleccionados
best_model = SVR(C=1000.0, shrinking=True, degree=2, gamma='auto')

# Ajustar el modelo SVM en el conjunto de entrenamiento más pequeño
best_model.fit(X_train_inner, y_train_inner)

# Apartado 6

En primer lugar utilizaremos nuestro mejor modelo hasta el momento para comprobar si las predicciones para valores altos son peores que para valores bajos (o viceversa). Esto lo haremos comparando su RSME

In [ ]:
from sklearn.metrics import mean_squared_error

# Predecir los valores de la variable objetivo para el conjunto de validación
y_pred = best_model.predict(X_val)

# Calcular el error cuadrático medio (RMSE)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE: {rmse}")

# Dividir las predicciones y los valores reales en grupos de altos y bajos
# Aquí, asumimos que "altos" son aquellos valores mayores a la mediana y "bajos" son los menores
median_energy = np.median(y_val)
y_val_high = y_val[y_val > median_energy]
y_val_low = y_val[y_val <= median_energy]
y_pred_high = y_pred[y_val > median_energy]
y_pred_low = y_pred[y_val <= median_energy]

# Calcular el RMSE para los valores altos y bajos
rmse_high = np.sqrt(mean_squared_error(y_val_high, y_pred_high))
rmse_low = np.sqrt(mean_squared_error(y_val_low, y_pred_low))

print(f"RMSE para valores altos: {rmse_high}")
print(f"RMSE para valores bajos: {rmse_low}")

# Comprobar si las predicciones para valores altos son peores que para valores bajos
if rmse_high > rmse_low:
    print("Las predicciones para valores altos son peores que para valores bajos.")
else:
    print("Las predicciones para valores altos no son necesariamente peores que para valores bajos.")

## Conversión a clasificación

In [ ]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# Eliminar la columna "datetime" del conjunto de datos
wind_ava.drop(["datetime"], axis=1, inplace=True)

# Determinar el tercer cuantil de la variable objetivo
energy_quantile = wind_ava['energy'].quantile(0.25)

# Clasificar los valores de energía como "baja" o "alta"
wind_ava['energy_class'] = ['baja' if e < energy_quantile else 'alta' for e in wind_ava['energy']]

# Convertir las etiquetas de clase a números
label_encoder = LabelEncoder()
wind_ava['energy_class_encoded'] = label_encoder.fit_transform(wind_ava['energy_class'])

# Almacenamos en una variable todas las columnas no relacionadas con la variable objetivo
excluir_columnas = ['energy', 'energy_class', 'energy_class_encoded']
numerical_vars = [col for col in wind_ava.columns if col not in excluir_columnas]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    wind_ava[numerical_vars], 
    wind_ava['energy_class_encoded'], 
    test_size=1/3, 
    random_state=100472166
)

# Ajustar el escalador a las características del conjunto de entrenamiento y transformarlas
scaler = StandardScaler()
X_train_class_scaled = scaler.fit_transform(X_train_class)
X_test_class_scaled = scaler.transform(X_test_class)

# Definir y entrenar un modelo de clasificación
classifier = RandomForestClassifier(random_state=100472166)
classifier.fit(X_train_class_scaled, y_train_class)

# Predecir las clases para el conjunto de prueba
y_pred_class = classifier.predict(X_test_class_scaled)

# Calcular la precisión del modelo de clasificación
accuracy = classifier.score(X_test_class_scaled, y_test_class)
print(f"Precisión del modelo de clasificación: {accuracy}")

## Hiperparámetros y Classification Report

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(
    wind_ava[numerical_vars], 
    wind_ava['energy_class_encoded'], 
    test_size=1/3, 
    random_state=100472166
)

# Definir el modelo
model = RandomForestClassifier(random_state=100472166)

# Definir los hiperparámetros a ajustar
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Realizar la búsqueda en cuadrícula para encontrar los mejores hiperparámetros
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Imprimir los mejores hiperparámetros
print("Mejores hiperparámetros: ", grid_search.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Predecir las clases para el conjunto de prueba
y_pred = best_model.predict(X_test)

# Calcular y mostrar las métricas
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

## Modelos distintos

Podemos probar a realizar el ajuste de hiperparámetros también para otros modelos diferentes al Random Forest Classifier, como SVC o los árboles de decisión

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Hiperparámetros para SVC
svc_params = {
    'C': np.logspace(-3, 3, 7),
    'gamma': np.logspace(-3, 3, 7),
    'shrinking': [True, False]
    # 'kernel': ['linear', 'rbf']
}

# Hiperparámetros para DecisionTreeClassifier
dt_params = {
    'max_depth': np.arange(1, 15),
    'min_samples_split': np.arange(2, 15),
    'min_samples_leaf': np.arange(1, 15),
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}

# Hiperparámetros para KNeighborsClassifier
knn_params = {
    'n_neighbors': np.arange(1, 15),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'leaf_size': np.arange(1, 15),
    'p': [1, 2]
}

# Ajustar el escalador a las características del conjunto de entrenamiento y transformarlas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# SVC
svc = SVC(random_state=42)
svc_search = GridSearchCV(svc, svc_params, cv=5, scoring='accuracy', n_jobs=-1)
svc_search.fit(X_train_scaled, y_train)
y_pred_svc = svc_search.predict(X_test_scaled)
print("Mejores hiperparámetros para SVC:", svc_search.best_params_)
print(classification_report(y_test, y_pred_svc))
print(confusion_matrix(y_test, y_pred_svc))
print("------------------------")

# DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt_search = GridSearchCV(dt, dt_params, cv=5, scoring='accuracy', n_jobs=-1)
dt_search.fit(X_train_scaled, y_train)
y_pred_dt = dt_search.predict(X_test_scaled)
print("Mejores hiperparámetros para DecisionTreeClassifier:", dt_search.best_params_)
print(classification_report(y_test, y_pred_dt))
print(confusion_matrix(y_test, y_pred_dt))
print("------------------------")

# KNeighborsClassifier
knn = KNeighborsClassifier()
knn_search = GridSearchCV(knn, knn_params, cv=5, scoring='accuracy', n_jobs=-1)
knn_search.fit(X_train_scaled, y_train)
y_pred_knn = knn_search.predict(X_test_scaled)
print("Mejores hiperparámetros para KNeighborsClassifier:", knn_search.best_params_)
print(classification_report(y_test, y_pred_knn))
print(confusion_matrix(y_test, y_pred_knn))
print("------------------------")

# Apartado 7